<div dir="rtl" style="width:90%; margin:auto;">
<p>هر دو کلاس <code>Debug</code> و <code>Trace</code>، کلاس‌های ایستا (static) هستند که قابلیت‌های پایه‌ای برای لاگ‌گیری و assertion فراهم می‌کنند.</p>

<p>تفاوت اصلی این دو کلاس در نوع استفاده مورد نظر آن‌ها است:<ul><li><strong>کلاس <code>Debug</code>:</strong> برای استفاده در بیلدهای دیباگ (debug builds) در نظر گرفته شده است.</li><li><strong>کلاس <code>Trace</code>:</strong> برای استفاده در هر دو نوع بیلد دیباگ و انتشار (release builds) در نظر گرفته شده است.</li></ul></p>

<p>هر فراخوانی که شما به متدهای <code>Debug</code> یا <code>Trace</code> انجام می‌دهید، توسط کامپایلر حذف می‌شود مگر اینکه نمادهای <code>DEBUG</code> یا <code>TRACE</code> تعریف شده باشند.</p>
</div>

Both the ***Debug*** and ***Trace*** classes provide `Write`, `WriteLine`, and `WriteIf` methods.

In [ ]:
Debug.Write ("Data");
Debug.WriteLine (23 * 34);
int x = 5, y = 3;
Debug.WriteIf (x > y, "x is greater than y");

The ***Trace*** class also provides the methods `TraceInformation`, `TraceWarning`, and `TraceError`.

### Fail and Assert

<div dir="rtl" style="width:90%; margin:auto;">
<p><strong>متد <code>Fail</code>:</strong></p>
<p>متد <code>Fail</code> پیامی را به هر <code>TraceListener</code> در مجموعه <code>Listeners</code> کلاس‌های <code>Debug</code> یا <code>Trace</code> ارسال می‌کند.</p>
<p>به صورت پیش‌فرض، این پیام به خروجی دیباگ نوشته می‌شود.</p>
</div>

In [ ]:
using System.Diagnostics;

Debug.Fail("File data.txt does not exist!");

<div dir="rtl" style="width:90%; margin:auto;">
<p><strong>متد <code>Assert</code>:</strong></p>
<p><p>متد <code>Assert</code> در صورتی که آرگومان بولی آن <code>false</code> باشد، متد <code>Fail</code> را فراخوانی می‌کند.</p></p>
</div>

In [3]:
using System.Diagnostics;
using System.IO;

Debug.Assert(File.Exists("data.txt"), "File data.txt does not exist!");

<div dir="rtl" style="width:90%; margin:auto;">
<p><strong>مجموعه <code>Listeners</code>:</strong></p>

<ul><li>هر دو کلاس <code>Debug</code> و <code>Trace</code> دارای مجموعه‌ای از <code>TraceListener</code> ها هستند که پیام‌های <code>Fail</code> و <code>Assert</code> را دریافت می‌کنند.</li><li>به صورت پیش‌فرض، این مجموعه شامل <code>DefaultTraceListener</code> است که پیام‌ها را به خروجی دیباگ می‌نویسد.</li><li>شما می‌توانید <code>TraceListener</code> های سفارشی به این مجموعه اضافه کنید تا پیام‌ها را به مکان‌های مختلفی مانند فایل‌ها، پایگاه‌های داده یا سیستم‌های لاگ‌گیری دیگر ارسال کنید.</li></ul>

</div>

The default listener has two key features:  
- When connected to a debugger such as `Visual Studio`, messages are written to the debug output window; otherwise, message content is ignored.
- When the `Fail` method is called (or an assertion fails), the `application is terminated`.

preddfined TraceLister :
- `TextWriterTraceListener` writes to a `Stream` or `TextWriter` or appends to a file.
- `EventLogTraceListener` writes to the Windows event log (Windows only).
- `EventProviderTraceListener` writes to the Event Tracing for Windows (ETW) subsystem (`cross-platform support`).

You can write ***owne*** `trace listeners` from scratch (by `subclassing TraceListener`)

In [ ]:
// Clear the default listener:
Trace.Listeners.Clear();

// Add a writer that appends to the trace.txt file:
Trace.Listeners.Add (new TextWriterTraceListener ("trace.txt"));

// Obtain the Console's output stream, then add that as a listener:
System.IO.TextWriter tw = Console.Out;
Trace.Listeners.Add (new TextWriterTraceListener (tw));

// Set up a Windows Event log source and then create/add listener.
// CreateEventSource requires administrative elevation, so this would
// typically be done in application setup.
if (!EventLog.SourceExists ("DemoApp")) // check exist EventLog with "DemoApp" name
    EventLog.CreateEventSource ("DemoApp", "Application"); // create if not exist
Trace.Listeners.Add (new EventLogTraceListener ("DemoApp")); // add to Listeners

In [4]:
TextWriterTraceListener tl = new TextWriterTraceListener(Console.Out);
        
        // تنظیم گزینه‌های خروجی لاگ برای شامل کردن تاریخ و زمان و پشته فراخوانی
        tl.TraceOutputOptions = TraceOptions.DateTime | TraceOptions.Callstack;
        
        // اضافه کردن TraceListener به مجموعه Listeners
        Trace.Listeners.Add(tl);
        
        // تنظیم سطح تورفتگی و اندازه تورفتگی
        Trace.IndentLevel = 1;
        Trace.IndentSize = 4;
        
        // ارسال یک پیام هشدار با استفاده از متد Trace
        Trace.TraceWarning("Orange alert");
        
        // تنظیم یک فیلتر برای فیلتر کردن پیام‌ها بر اساس نوع رویداد
        tl.Filter = new EventTypeFilter(SourceLevels.Warning);
        
        // ارسال یک پیام اطلاعاتی که توسط فیلتر مسدود می‌شود
        Trace.TraceInformation("This message will not be displayed because of the filter.");

 // خروجی در visual studio قابل مشاهده است

### Flushing and Closing Listeners

<div dir="rtl" style="width:90%; margin:auto;">

<p>برخی شنونده‌ها مانند <code>TextWriterTraceListener</code> به جریان‌هایی می‌نویسند که به کش (cache) وابسته هستند.</p>

<p>این دو پیامد دارد:<ul><li>یک پیام ممکن است بلافاصله در جریان خروجی یا فایل ظاهر نشود.</li><li>شما باید قبل از پایان برنامه، شنونده را ببندید یا حداقل فلاش کنید؛ در غیر این صورت، محتوای موجود در کش (که تا 4 کیلوبایت به طور پیش‌فرض می‌تواند باشد) از دست می‌رود.</li></ul></p>

<p><p><strong>متدهای <code>Close</code> و <code>Flush</code>:</strong></p><ul><li>کلاس‌های <code>Trace</code> و <code>Debug</code> متدهای استاتیک <code>Close</code> و <code>Flush</code> را فراهم می‌کنند که به ترتیب <code>Close</code> یا <code>Flush</code> را بر روی تمام شنونده‌ها فراخوانی می‌کنند. این متدها به نوبه خود <code>Close</code> یا <code>Flush</code> را بر روی هر نویسنده یا جریان زیرین فراخوانی می‌کنند.</li><li>متد <code>Close</code> به صورت ضمنی <code>Flush</code> را فراخوانی می‌کند، دسته‌های فایل را می‌بندد و مانع از نوشتن داده‌های بیشتر می‌شود.</li></ul></p>
</div>